In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.311365,"""a""",0.452591,0.634948,0.550164,0.931355,0.952016,-0.566861,0.289622
0.841471,1,"""a""",1,0.8277,"""a""",0.270705,0.515903,0.450484,0.386854,0.178172,-0.480284,0.222374
0.909297,2,"""a""",0,0.550424,"""a""",0.021199,0.841007,0.478687,0.316904,0.450743,-0.462496,0.333436
0.14112,3,"""a""",1,0.179424,"""a""",0.705329,0.564284,0.305577,0.816216,0.072691,-0.183216,0.315091
-0.756802,4,"""a""",1,0.863432,"""a""",0.388277,0.377642,0.819863,0.659511,0.449105,-1.058251,0.138485


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.931355,-0.952016
-0.841471,-0.386854,-0.178172
-0.909297,-0.316904,-0.450743
-0.14112,-0.816216,-0.072691
0.756802,0.271843,0.502911


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

lstsq_coeffs
list[f64]
"[-0.503852, -0.336489]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("lstsq_coeffs")

y,y2
list[f64],list[f64]
"[-0.503852, -0.336489]","[0.086448, 0.407609]"


In [9]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.215761,0.001671,129.134454,0.0,0.212486,0.219036
"""exp(x2)""",0.174188,0.000675,258.183738,0.0,0.172866,0.175511
"""sin(x3)""",-1.743341,0.001338,-1302.513632,0.0,-1.745965,-1.740717
"""__bias__""",-0.105863,0.001501,-70.519428,0.0,-0.108806,-0.10292


In [10]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,lstsq_coeffs
str,list[f64]
"""a""","[-0.492431, -0.34378]"
"""a""","[-0.492431, -0.34378]"
"""a""","[-0.492431, -0.34378]"
"""a""","[-0.492431, -0.34378]"
"""a""","[-0.492431, -0.34378]"
…,…
"""b""","[-0.515136, -0.329358]"
"""b""","[-0.515136, -0.329358]"
"""b""","[-0.515136, -0.329358]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.452591,0.634948,-0.566861,-0.441692,-0.125169
0.270705,0.515903,-0.480284,-0.309991,-0.170293
0.021199,0.841007,-0.462496,-0.293671,-0.168825
0.705329,0.564284,-0.183216,-0.545257,0.362041
0.388277,0.377642,-1.058251,-0.322706,-0.735545


In [12]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,lstsq_coeffs
str,list[f64]
"""b""","[-0.515136, -0.329358]"
"""a""","[-0.492431, -0.34378]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,lstsq_coeffs
str,list[f64]
"""b""","[-0.348425, -0.153208]"
"""a""","[-0.327031, -0.166565]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.532845
"""b""",-0.530666


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.566861,0.452591,0.634948,null,null
-0.480284,0.270705,0.515903,null,null
-0.462496,0.021199,0.841007,null,null
-0.183216,0.705329,0.564284,null,null
-1.058251,0.388277,0.377642,"[-0.32733, -0.639846]",-0.368727
…,…,…,…,…
-0.741058,0.126328,0.802046,"[-1.287848, -0.482204]",-0.549441
-0.394902,0.328996,0.55712,"[0.338738, -0.881164]",-0.37947
-0.269907,0.80369,0.067989,"[-0.200837, -0.749749]",-0.212386


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[29.13704, 28.802353, 28.740907]"


In [18]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
29.022991,"[0.651065, 0.759022]"
28.750397,"[0.759022, -0.651065]"


In [19]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
0.620102
-0.321769
-0.160422
-0.122288
0.061394


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.328758,0.9815,0.517283,0.505697,0.511425,0.515614,0.509259
"""a""",0.330871,0.997721,0.507475,0.501798,0.50462,0.504864,0.505687


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""world"""
"""to"""
"""hello"""
"""going"""
"""church"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""go"""
"""hello"""
"""church"""
"""world"""
""""""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [30]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [31]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [32]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.024512
-1.297304
1.30328


In [33]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,-0.597865,null
null,0.809882,null
-0.024512,1.264788,0.407689
-1.297304,1.358113,0.737675
1.30328,2.011592,-0.108937


In [34]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""Ybxn""",null
null,"""T5bT""",null
-0.024512,"""jxBx""","""9Hy"""
-1.297304,"""LAl""","""mH"""
1.30328,"""0""","""vgyA"""


In [35]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.024512,"""Q8EN8"""
-1.297304,"""FWpIg"""
1.30328,"""ELvq3"""


In [36]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,0.594895,null,0.595009
null,0.48421,null,0.484496
-0.024512,-0.843753,1.69616,-0.843997
-1.297304,-0.852032,2.117996,-0.851554
1.30328,1.629613,1.94899,1.629544


In [37]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.211779,0.216642,2
null,0.368254,0.707741,8
-0.024512,0.813206,0.627686,3
-1.297304,0.397062,1.675937,3
1.30328,0.743291,-1.042864,5


In [38]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-0.042597,0.966029,0.207466,0.901466


In [39]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.463582,0.757397,1,6
1,0.191577,0.786679,2,1
2,0.958019,0.222917,3,2
0,0.790611,0.042357,3,4
1,0.644218,0.040001,4,4


In [40]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-0.184305,0.853778}","{40.942411,0.262527}","{0.535979,0.709315}"


In [41]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {0.853737,0.393312}  ┆ {34.980909,0.516909} ┆ {0.028318,0.998454} │
│ 1         ┆ {-0.680579,0.496185} ┆ {40.07506,0.294186}  ┆ {1.038676,0.385823} │
│ 2         ┆ {-0.49192,0.622808}  ┆ {35.837897,0.476251} ┆ {0.325277,0.861141} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [42]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
564,0.1128
577,0.1154
536,0.1072
524,0.1048
547,0.1094
563,0.1126
559,0.1118
550,0.11
580,0.116


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [43]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [44]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.129931,0.973586,0.470911,0.385868,3.318237,13
1,0.800518,0.507734,0.297454,0.442696,5.756227,14
2,0.145785,0.794784,0.574881,0.417187,9.902383,21
3,0.907533,0.555578,0.561712,0.83716,8.144571,21
4,0.267012,0.49415,0.124488,0.468267,3.477004,19


In [45]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.129931,0.973586,0.470911,0.385868,3.318237,86
1,0.800518,0.507734,0.297454,0.442696,5.756227,209
2,0.145785,0.794784,0.574881,0.417187,9.902383,166
3,0.907533,0.555578,0.561712,0.83716,8.144571,843
4,0.267012,0.49415,0.124488,0.468267,3.477004,221


In [46]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.129931,0.973586,0.470911,0.385868,3.318237,"[0, 200, … 984]"
1,0.800518,0.507734,0.297454,0.442696,5.756227,"[1, 283, … 1815]"
2,0.145785,0.794784,0.574881,0.417187,9.902383,"[2, 905, … 7]"
3,0.907533,0.555578,0.561712,0.83716,8.144571,"[3, 566, … 568]"
4,0.267012,0.49415,0.124488,0.468267,3.477004,"[4, 1018, … 1962]"


In [47]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 200, … 1833]  ┆ 7                  │
│ 1   ┆ [1, 283, … 1913]  ┆ 7                  │
│ 2   ┆ [2, 905, … 307]   ┆ 10                 │
│ 3   ┆ [3, 566, … 1347]  ┆ 12                 │
│ 4   ┆ [4, 1018, … 1890] ┆ 11                 │
└─────┴───────────────────┴────────────────────┘


In [48]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.129931,0.973586,0.470911,0.385868,3.318237,"[0, 200, … 984]","[0.0, 0.041923, … 0.068588]"
1,0.800518,0.507734,0.297454,0.442696,5.756227,"[1, 283, … 1815]","[0.0, 0.047633, … 0.071768]"
2,0.145785,0.794784,0.574881,0.417187,9.902383,"[2, 905, … 7]","[0.0, 0.061459, … 0.063645]"
3,0.907533,0.555578,0.561712,0.83716,8.144571,"[3, 566, … 568]","[0.0, 0.030469, … 0.044061]"
4,0.267012,0.49415,0.124488,0.468267,3.477004,"[4, 1018, … 1962]","[0.0, 0.029224, … 0.069832]"


In [49]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
1,0.800518,0.507734,0.297454,0.442696,5.756227
3,0.907533,0.555578,0.561712,0.83716,8.144571
4,0.267012,0.49415,0.124488,0.468267,3.477004
8,0.655189,0.462989,0.108567,0.412206,3.517657
9,0.300874,0.75718,0.7756,0.251852,3.904736


In [50]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
19,0.502213,0.51037,0.02334,0.530426,0.013364
37,0.543824,0.421768,0.874612,0.294588,6.3733
65,0.513678,0.434413,0.815825,0.98528,4.956589
93,0.577378,0.501789,0.340534,0.373521,1.96916
95,0.491906,0.471299,0.389033,0.689185,8.74063


In [51]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
95,0.491906,0.471299,0.389033,0.689185,8.74063
298,0.495451,0.526295,0.594563,0.286831,9.338581
339,0.544305,0.518736,0.010804,0.891533,9.958304
354,0.491961,0.496793,0.121755,0.258258,4.933692
607,0.495183,0.496355,0.443438,0.90197,7.748199


In [52]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1759, … 1295]",5
1,"[1, 96, 735]",3
2,"[2, 1557, … 620]",6
3,"[3, 916, … 626]",4
4,"[4, 192, … 1259]",4


# Using PDS Expressions On Series / NumPy arrays

In [53]:
df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
)
df.head()

actual,predicted,0-2,0-9
f64,f64,i32,i32
0.0,0.196065,2,2
1.0,0.772261,0,2
0.0,0.125342,2,8
1.0,0.628726,2,1
1.0,0.261299,0,5


In [54]:
pds.eval_series(
    df["0-2"], df["0-9"], # use series as args
    expr = "jaccard_col" # name of the pds expression
)

jaccard_col
f64
0.3


In [55]:
pds.eval_series(
    df["actual"], df["predicted"], # use series as args
    expr = "query_binary_metrics" # name of the pds expression
).unnest("binary_metrics")

precision,recall,f,average_precision,roc_auc
f64,f64,f64,f64,f64
0.500951,0.498566,0.499755,0.499112,0.497159


In [56]:
pds.eval_series(
    # can also use NumPy
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    expr = "psi", # name of the pds expression
    n_bins = 5, 
    return_report = True
).unnest("psi")

cnt<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.201088,0.2,0.205,0.000123
0.397245,0.2,0.177,0.00281
0.590998,0.2,0.204,0.000079
0.785837,0.2,0.215,0.001085
inf,0.2,0.199,0.000005


In [57]:
pds.eval_series(
    np.random.random(size = 1000), 
    expr = "query_cid_ce", # name of the pds expression
)

cid_ce
f64
13.344001


In [58]:
pds.eval_series(
    pl.Series(values=np.random.random(size = 1000)), 
    expr = "query_c3_stats", # name of the pds expression
    lag = 3
)

c3_stats
f64
0.106919
